# Script for creating word-order manipulation datasets from the Pereira2018 fMRI stimuli

In [1]:
import re
from pathlib import Path
from os.path import abspath
import os
import numpy as np
import random
import pickle
import csv
import subprocess

In [2]:
print(os.getcwd())

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/ressources/stimuli_creation


In [3]:
importpath = abspath('../..')
os.chdir(importpath)
print(os.getcwd())

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp


In [4]:
#add seeds for reproducability
np.random.seed(42)
random.seed(42)

# Load base stimulus dataframe (Pereira 2018)

In [5]:
from neural_nlp.benchmarks.neural import *
import neural_nlp
from neural_nlp.stimuli import StimulusSet
import xarray as xr

benchmark_pool = [
    # primary benchmarks
    ('Pereira2018-encoding', PereiraEncoding),
]
benchmark_pool = {identifier: LazyLoad(lambda identifier=identifier, ctr=ctr: ctr(identifier=identifier))
                  for identifier, ctr in benchmark_pool}

# fetch stimulus set
benchmark = benchmark_pool['Pereira2018-encoding']
stimuli_df = benchmark._target_assembly.attrs['stimulus_set']
stimuli_df

Loading lookup from /om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_collection/lookup.csv
/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/neural_nlp/../ressources/stimuli


 We're running in the NEW version of the implementations.py script.




/om2/user/ckauf/anaconda/envs/perturbedenv/lib/python3.6/site-packages/brainio_base/assemblies.py:224: FutureWarning: The inplace argument has been deprecated and will be removed in a future version of xarray.
  xr_data.set_index(append=True, inplace=True, **coords_d)


,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,Beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,It is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,"As a passive form of agriculture, it does not ...",2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,Beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,Artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping
...,...,...,...,...,...,...,...,...
622,Some windows have multiple panes to increase i...,379,384sentences.379,384sentences,384sentences.building_part,95,Window,building_part
623,A woman is a female human adult.,380,384sentences.380,384sentences,384sentences.human,96,Woman,human
624,A woman is stereotypically seen as a caregiver.,381,384sentences.381,384sentences,384sentences.human,96,Woman,human
625,A woman can become pregnant and bear children.,382,384sentences.382,384sentences,384sentences.human,96,Woman,human


In [6]:
# benchmark content
benchmark._target_assembly.values.shape
benchmark._target_assembly.coords

Coordinates:
  * presentation      (presentation) MultiIndex
  - stimulus_num      (presentation) int64 0 0 1 1 2 2 3 ... 12 12 13 13 14 14
  - passage_index     (presentation) int64 1 1 1 1 1 1 1 1 2 ... 3 4 4 4 4 4 4 4
  - passage_label     (presentation) object 'Accordion' ... 'dreams'
  - passage_category  (presentation) object 'music' 'beekeeping' ... 'dreams'
  - stimulus_id       (presentation) object '384sentences.0' ... '243sentences.14'
  - story             (presentation) object '384sentences.music' ... '243sentences.dreams'
  - experiment        (presentation) object '384sentences' ... '243sentences'
  * neuroid           (neuroid) MultiIndex
  - subject           (neuroid) object '018' '018' '018' ... '018' '018' '018'
  - voxel_num         (neuroid) int64 28 29 31 32 38 42 ... 152 153 154 159 160
  - atlas             (neuroid) object 'language' 'language' ... 'language'
  - filter_strategy   (neuroid) object '' '' '' '' '' '' ... '' '' '' '' '' ''
  - atlas_selection   (

# Create different perturbed versions of the benchmark

In [7]:
stimuli_path = os.path.join(os.getcwd(),'ressources/stimuli_creation')
os.chdir(stimuli_path)
print(os.getcwd())

savedir = abspath('../scrambled_stimuli_dfs')
print(savedir)
os.makedirs(savedir, exist_ok=True)

/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/ressources/stimuli_creation
/rdma/vast-rdma/vast/cpl/ckauf/perturbed-neural-nlp/ressources/scrambled_stimuli_dfs


# Scrambling | Mollica et al. (2020)

## 1. Create stable PMI scrambled dataset versions (Mollica et al. 2020)

`The below cell calls a python script with input stimuli files already hardcoded!`

In [8]:
! python permute_sentences.py

stim_243sentences
0	beekeeping encourages the conservation of local habitats
1	beekeeping encourages the conservation of habitats local
2	encourages beekeeping the conservation of habitats local
3	beekeeping conservation encourages the of habitats local
4	conservation beekeeping encourages the of habitats local
5	conservation beekeeping encourages the habitats of local
6	conservation encourages beekeeping of the habitats local
7	beekeeping conservation of encourages habitats local the
0	it is in every beekeeper's interest to conserve local plants that produce pollen
1	it is in every beekeeper's interest to conserve local plants produce that pollen
2	it in is every beekeeper's interest to conserve local plants produce that pollen
3	in it is every beekeeper's interest to conserve local plants produce that pollen
4	in it is every beekeeper's interest to conserve local plants produce pollen that
5	in it is every interest beekeeper's to conserve local plants produce pollen that
6	in it is i

2	even i've never done any of the reading assignments
3	even i've never done of any the reading assignments
4	even i've never of done any the reading assignments
5	even done i've never of any the reading assignments
6	even done i've never of any reading the assignments
7	even done i've of never any reading the assignments
0	i know many other people who have the same nightmare
1	i many know other people who have the same nightmare
2	many i know other people who have the same nightmare
3	many i know other who people have the same nightmare
4	many i know who other people have the same nightmare
5	many other i know who people have the same nightmare
6	many other i know who people have the nightmare same
7	many other i know who people the have nightmare same
0	each morning participants in the study had to write down their dream experience from the previous night
1	each morning participants the in study had to write down their dream experience from the previous night
2	morning each participa

7	on the good data and social economic effects of legalized gambling hard are to by come
0	some studies indicate that having a casino nearby makes gambling problems more likely
1	some studies indicate that having a casino nearby makes gambling more problems likely
2	some indicate studies that having a casino nearby makes gambling more problems likely
3	indicate some studies that having a casino nearby makes gambling more problems likely
4	indicate some studies that having a casino nearby makes gambling more likely problems
5	indicate some studies that a having casino nearby makes gambling more likely problems
6	indicate some studies a that having casino nearby makes gambling more likely problems
7	indicate that some studies a having casino nearby gambling makes problems likely more
0	gambling may also be associated with personal bankruptcies and marriage problems
1	gambling also may be associated with personal bankruptcies and marriage problems
2	also gambling may be associated with pe

4	storms the strongest deposited layers of windblown beach sand at bottoms of the coastal lakes
5	storms the strongest deposited of layers windblown beach sand at bottoms of the coastal lakes
6	storms the strongest of deposited layers windblown beach sand at bottoms of the coastal lakes
7	storms deposited the strongest of layers windblown beach at sand the of bottoms coastal lakes
0	trees drenched by passing hurricanes preserve a record of each storm in their growth rings
1	trees drenched by passing hurricanes preserve a record of each in storm their growth rings
2	trees by drenched passing hurricanes preserve a record of each in storm their growth rings
3	by trees drenched passing hurricanes preserve a record of each in storm their growth rings
4	by trees drenched passing hurricanes preserve a record of each in their storm growth rings
5	by trees drenched passing preserve hurricanes a record of each in their storm growth rings
6	by trees drenched preserve passing hurricanes a record o

7	blades underneath the sharp lawn the mower precisely each sever blade grass of
0	mowing the lawn is a long task and it has taught me patience
1	mowing the lawn is a long task and it has me taught patience
2	mowing lawn the is a long task and it has me taught patience
3	lawn mowing the is a long task and it has me taught patience
4	lawn mowing the is a long task and it has me patience taught
5	lawn mowing the is long a task and it has me patience taught
6	lawn mowing the long is a task and it has me patience taught
7	lawn is mowing the long a task and has it taught patience me
0	a lawn mower is a machine with revolving blades that cut grass to an even height
1	a lawn mower is a machine with revolving blades that grass cut to an even height
2	a mower lawn is a machine with revolving blades that grass cut to an even height
3	mower a lawn is a machine with revolving blades that grass cut to an even height
4	mower a lawn is a machine with revolving blades that grass to cut an even height


...

4	insects spiders are legs with eight that make silk
5	insects with spiders are legs eight that make silk
6	insects with spiders are legs eight make that silk
7	insects with spiders legs are eight make that silk
0	spiders build webs with silk and use them to catch insects
1	spiders webs build with silk and use them to catch insects
2	webs spiders build with silk and use them to catch insects
3	webs spiders build with and silk use them to catch insects
4	webs spiders build and with silk use them to catch insects
5	webs with spiders build and silk use them to catch insects
6	webs with spiders build and silk use them catch to insects
7	webs spiders with build and silk them use catch insects to
0	spiders use venomous fangs to paralyse their stuck prey
1	spiders venomous use fangs to paralyse their stuck prey
2	venomous spiders use fangs to paralyse their stuck prey
3	venomous spiders use fangs paralyse to their stuck prey
4	venomous spiders use paralyse fangs to their stuck prey
5	venomous

1	they are used to play videogames when connected to a console game
2	they used are to play videogames when connected to a console game
3	used they are to play videogames when connected to a console game
4	used they are to videogames play when connected to a console game
5	used they are videogames to play when connected to a console game
6	used to they are videogames play when connected to a console game
7	used they to are videogames play connected when a game to console
0	television content can be broadcast or received via cable or internet
1	television can content be broadcast or received via cable or internet
2	can television content be broadcast or received via cable or internet
3	can television content be or broadcast received via cable or internet
4	can television content or be broadcast received via cable or internet
5	can be television content or broadcast received via cable or internet
6	can be television content or broadcast received via or cable internet
7	can television be 

7	is a a train series connected of cars moves that between on stations a track
0	the train is pulled by a locomotive powered by diesel or electricity
1	the train is pulled by a locomotive powered by diesel electricity or
2	the is train pulled by a locomotive powered by diesel electricity or
3	is the train pulled by a locomotive powered by diesel electricity or
4	is the train pulled a by locomotive powered by diesel electricity or
5	is the train a pulled by locomotive powered by diesel electricity or
6	is pulled the train a by locomotive powered by diesel electricity or
7	is the pulled train a by powered locomotive diesel or by electricity
0	a freight train transports containers of goods or fuel over long distances
1	a freight train transports containers of goods or fuel over distances long
2	a train freight transports containers of goods or fuel over distances long
3	train a freight transports containers of goods or fuel over distances long
4	train a freight transports of containers go

1	vultures are scavenging birds feeding mostly on the carcasses of animals dead
2	vultures scavenging are birds feeding mostly on the carcasses of animals dead
3	scavenging vultures are birds feeding mostly on the carcasses of animals dead
4	scavenging vultures are birds mostly feeding on the carcasses of animals dead
5	scavenging vultures are mostly birds feeding on the carcasses of animals dead
6	scavenging birds vultures are mostly feeding on the carcasses of animals dead
7	scavenging vultures birds are mostly feeding the on of dead carcasses animals
0	many vultures have a bald head without feathers for losing heat
1	many have vultures a bald head without feathers for losing heat
2	have many vultures a bald head without feathers for losing heat
3	have many vultures a head bald without feathers for losing heat
4	have many vultures head a bald without feathers for losing heat
5	have a many vultures head bald without feathers for losing heat
6	have a many vultures head bald without fea

0	whales have been hunted for meat whale oil and ambergris
1	whales been have hunted for meat whale oil and ambergris
2	been whales have hunted for meat whale oil and ambergris
3	been whales have hunted meat for whale oil and ambergris
4	been whales have meat hunted for whale oil and ambergris
5	been hunted whales have meat for whale oil and ambergris
6	been hunted whales have meat for whale oil ambergris and
7	been hunted whales have meat for oil whale ambergris and
0	a window is a transparent opening in a wall to let light through
1	a window is a transparent opening in a wall to light let through
2	a is window a transparent opening in a wall to light let through
3	is a window a transparent opening in a wall to light let through
4	is a window a transparent opening in a wall to light through let
5	is a window a opening transparent in a wall to light through let
6	is a window opening a transparent in a wall to light through let
7	is a a window opening transparent in a to wall let throug

`filter out conditions used in Mollica et al. (2020)`

In [9]:
def create_scr_lists(filename):
    with open(os.path.join(stimuli_path,filename),"r") as f:
        reader = csv.reader(f, delimiter="\t")
        sentences = list(reader)
    Original = [sentence[1] + '.' for sentence in sentences if int(sentence[0]) == 0]
    Scr1 = [sentence[1] + '.' for sentence in sentences if int(sentence[0]) == 1]
    Scr3 = [sentence[1] + '.' for sentence in sentences if int(sentence[0]) == 3]
    Scr5 = [sentence[1] + '.' for sentence in sentences if int(sentence[0]) == 5]
    Scr7 = [sentence[1] + '.' for sentence in sentences if int(sentence[0]) == 7]
    
    return(Original, Scr1, Scr3, Scr5, Scr7)

In [10]:
Original_243, Scr1_243, Scr3_243, Scr5_243, Scr7_243 = create_scr_lists("stim_243sentences_scrambled.txt")
Original_384, Scr1_384, Scr3_384, Scr5_384, Scr7_384 = create_scr_lists("stim_384sentences_scrambled.txt")

Original = Original_243 + Original_384
Scr1 = Scr1_243 + Scr1_384
Scr3 = Scr3_243 + Scr3_384
Scr5 = Scr5_243 + Scr5_384
Scr7 = Scr7_243 + Scr7_384

In [11]:
condition_names = ["Original", "Scr1", "Scr3", "Scr5", "Scr7"]
condition_lists = [Original, Scr1, Scr3, Scr5, Scr7]
condition_dict = dict(zip(condition_names, condition_lists))
condition_dict

{'Original': ['beekeeping encourages the conservation of local habitats.',
  "it is in every beekeeper's interest to conserve local plants that produce pollen.",
  'as a passive form of agriculture it does not require that native vegetation be cleared to make way for crops.',
  'beekeepers also discourage the use of pesticides on crops because they could kill the honeybees.',
  'artisanal beekeepers go to extremes for their craft but their product is worth the effort.',
  'artisanal honey-making emphasizes quality and character over quantity and consistency.',
  'to produce the finest honey beekeepers become micromanagers of their honeybees.',
  'they scout the fields know when nectar flows and select the best ways to extract honey.',
  'as the beekeeper opens the hive the deep hum of 40000 bees fills the air.',
  'the beekeeper checks honey stores pollen supplies and the bee nursery.',
  "bees crawl across his bare arms and hands but they don't sting because they're gentle.",
  "i hav

`save xarrays with scrambled stimulus sets`

In [12]:
def make_datasets(scrambled_sentence_list, cond):
    """
    Input: text file for each scrambling condition
    Output: Saves pickle files for each condition
    (same xarray just different stimulus_set!)
    """
    stimuli_scrambled = stimuli_df.copy()
    stimuli_scrambled["sentence"] = scrambled_sentence_list

    fname = f"{savedir}/stimuli_{cond}.pkl"
    with open(fname, 'wb') as fout:
        pickle.dump(stimuli_scrambled, fout)

for cond, scrambled_sentence_list in condition_dict.items():
    make_datasets(scrambled_sentence_list, cond)
print("Done!")

Done!


## 2. Create low PMI scrambled dataset version (Mollica et al. 2020)

In [13]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ckauf/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/ckauf/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
stimuli_files = ["stim_243sentences_nopunct.txt", "stim_384sentences_nopunct.txt"]

def create_low_pmi(stimuli_files):
    
    content_regex = re.compile('JJ.*|NN.*|RB.*|VB.*')
    #JJ = adjectives
    #NN = nouns
    #RB = adverbs
    #VB = verbs
    
    lowPMI = []

    for filename in stimuli_files:
        with open(os.path.join(stimuli_path,filename), "r") as f:
            sentences = f.readlines()
            sentences = [x.strip().lower() for x in sentences]

        for sent in sentences:
            words = re.split(r'\s+', sent) #don't use NLTK word tokenizer, or else build work-around for 's sentences
            pos_tagged = nltk.pos_tag(words)

            content = [x[0] for x in pos_tagged if re.match(content_regex,x[1])]
            function = [x[0] for x in pos_tagged if not re.match(content_regex,x[1])]

            content1 = []
            content2 = []
            for i, c in enumerate(content): #create two lists of content words (even and odd indexed content words in the sentence)
                if i % 2 == 0:
                    content1.append(c)
                else:
                    content2.append(c)
            function1 = []
            function2 = []
            for i, c in enumerate(function): #create two lists of function words (even and odd indexed function words in the sentence)
                if i % 2 == 0:
                    function1.append(c)
                else:
                    function2.append(c)
            lowPMI.append(' '.join(content1 + function1 + function2 + content2)+".")

    stimuli_lowPMI = stimuli_df.copy()
    stimuli_lowPMI["sentence"] = lowPMI

    fname = f"{savedir}/stimuli_lowPMI.pkl"
    with open(fname, 'wb') as fout:
        pickle.dump(stimuli_lowPMI, fout)

    return lowPMI

lowPMI = create_low_pmi(stimuli_files)
print("Done!")

Done!


## 3. Create random word-list dataset version (Mollica et al. 2020)

In [15]:
stim_243 = os.path.join(stimuli_path, 'stim_243sentences_nopunct.txt')
stim_384 = os.path.join(stimuli_path, 'stim_384sentences_nopunct.txt')

In [16]:
all_words = []
lengths = []

for l in open(stim_243, 'r'):
    l = l.strip().lower()
    
    words = re.split(r'\s+', l)
    lengths.append(len(words))
    all_words += words

for l in open(stim_384, 'r'):
    l = l.strip().lower()
    
    words = re.split(r'\s+', l)
    lengths.append(len(words))
    all_words += words

print(len(all_words))
print(len(lengths))
random.shuffle(all_words)

random_sentences = []
for n in lengths:
    random_sample = random.sample(all_words,n)
    new_sentence = ' '.join(random_sample) + '.'
    random_sentences.append(new_sentence)
    
    [all_words.remove(elm) for elm in random_sample] #remove selected words from list
    #print(len(all_words))
    
assert len(all_words) == 0, f"Not all words from the dataset have been used. Length of word list is {len(all_words)}!"

print(random_sentences[:10])

stimuli_random = stimuli_df.copy()
stimuli_random["sentence"] = random_sentences

fname = f"{savedir}/stimuli_random.pkl"
with open(fname, 'wb') as fout:
    pickle.dump(stimuli_random, fout)

7730
627
['three climb around the the spears vapor.', 'of shears metallic is in individual machine for fracture a singer can have.', "insulation body used i've control a fever shark a side crop humble pliers treated and places beekeeper large characteristic own.", 'stop stone the act are flood export and genetic cultures edge varieties reptiles used people.', 'polar almost a a she crime furniture many have the as taste if a of.', 'the sauce an hammers of log driving to and let.', 'was musical of the at that most apply ice swamp of.', 'cabin to meant were the drop glass objects the night the heads for hand same orchestra.', 'or in a locks of to can a the to four instantly replacement are things.', 'patients a plant is the regions small evidence eye bone solid.']


## 4. Create backward dataset version (control for Mollica et al. 2020)

In [17]:
backward_stim = []

for l in open(stim_243, 'r'):
    l = l.strip().lower()
    
    words = re.split(r'\s+', l)
    new_words = words[::-1]
    new_sent = ' '.join(new_words) + '.'
    backward_stim.append(new_sent)

for l in open(stim_384, 'r'):
    l = l.strip().lower()
    
    words = re.split(r'\s+', l)
    new_words = words[::-1]
    new_sent = ' '.join(new_words) + '.'
    backward_stim.append(new_sent)

print(len(backward_stim))
print(words)
print(new_words)
print(new_sent)

stimuli_backward = stimuli_df.copy()
stimuli_backward["sentence"] = backward_stim

fname = f"{savedir}/stimuli_backward.pkl"
with open(fname, 'wb') as fout:
    pickle.dump(stimuli_backward, fout)
stimuli_backward

627
['a', 'woman', 'has', 'different', 'reproductive', 'organs', 'than', 'a', 'man']
['man', 'a', 'than', 'organs', 'reproductive', 'different', 'has', 'woman', 'a']
man a than organs reproductive different has woman a.


,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,habitats local of conservation the encourages ...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,pollen produce that plants local conserve to i...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,crops for way make to cleared be vegetation na...,2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,honeybees the kill could they because crops on...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,effort the worth is product their but craft th...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping
...,...,...,...,...,...,...,...,...
622,inside insulation increase to panes multiple h...,379,384sentences.379,384sentences,384sentences.building_part,95,Window,building_part
623,adult human female a is woman a.,380,384sentences.380,384sentences,384sentences.human,96,Woman,human
624,caregiver a as seen stereotypically is woman a.,381,384sentences.381,384sentences,384sentences.human,96,Woman,human
625,children bear and pregnant become can woman a.,382,384sentences.382,384sentences,384sentences.human,96,Woman,human


In [18]:
backward_stim[:5]

['habitats local of conservation the encourages beekeeping.',
 "pollen produce that plants local conserve to interest beekeeper's every in is it.",
 'crops for way make to cleared be vegetation native that require not does it agriculture of form passive a as.',
 'honeybees the kill could they because crops on pesticides of use the discourage also beekeepers.',
 'effort the worth is product their but craft their for extremes to go beekeepers artisanal.']

## 5. Create random LowPMI dataset version (control for Mollica et al. 2020)

### requires OM

`go to folder "add_random_lowPMI_condition" & follow instructions there.`